# Shortest Path Calculation with PostGIS and pgRouting
This notebook demonstrates how to find and visualize the shortest route between two points using a `roads` table with `id` and `geom` (LineString) columns in PostGIS. Both A* and Dijkstra algorithms are used and compared visually.

# Geospatial Data Processing with PostGIS
Learn how to use PostGIS for spatial data operations, shortest path calculations, and route visualization.

## 1. Connect to PostgreSQL/PostGIS
Set up the database connection using your credentials.

In [12]:
import psycopg2
conn = psycopg2.connect(dbname="postgres", user="postgres", password="mysecretpassword", host="localhost", port="5435")
cur = conn.cursor()

## 2. Define Start and End Points
Specify the coordinates for the start and end locations. These will be used to find the nearest road segments and calculate the route.

In [13]:
# Example coordinates for start and end points
start_lon, start_lat = 32.7347912, 39.954748
end_lon, end_lat = 32.7576441, 39.9498783

## 3. Find Nearest Road Segments
Find the nearest road segment to the start and end points using spatial queries. This ensures the route calculation starts and ends at the closest available network segment.

In [14]:
# Find nearest road segment to start point
# Create roads table from planet_osm_line if it doesn't exist
cur.execute("""
    CREATE TABLE IF NOT EXISTS roads AS 
    SELECT osm_id as id, way as geom 
    FROM planet_osm_line 
    WHERE highway IS NOT NULL;
""")
cur.execute("CREATE INDEX IF NOT EXISTS roads_geom_idx ON roads USING GIST(geom);")
conn.commit()

# Find nearest road segment to start point
cur.execute("""SELECT id FROM roads ORDER BY geom <-> ST_SetSRID(ST_MakePoint(%s, %s), 4326) LIMIT 1;""", (start_lon, start_lat))
start_road_id = cur.fetchone()[0]
# Find nearest road segment to end point
cur.execute("""SELECT id FROM roads ORDER BY geom <-> ST_SetSRID(ST_MakePoint(%s, %s), 4326) LIMIT 1;""", (end_lon, end_lat))
end_road_id = cur.fetchone()[0]
print('Start road id:', start_road_id)
print('End road id:', end_road_id)

conn.commit()

Start road id: 710513863
End road id: 447747710


## 4. Prepare Edges Table for pgRouting
pgRouting requires an edges table with source and target node IDs. If your `roads` table does not have these, you need to create them. This step ensures the network is topologically correct for routing algorithms.

In [15]:
# Check if edges_custom table exists, if not create it
cur.execute("""
   SELECT EXISTS (
      SELECT FROM information_schema.tables 
      WHERE table_name = 'edges_custom'
   );
""")
edges_exists = cur.fetchone()[0]

if not edges_exists:
   print("--- Creating 'edges_custom' table ---")
   cur.execute("""
   CREATE TABLE edges_custom AS
   SELECT osm_id as id, way as geom FROM planet_osm_line WHERE highway IS NOT NULL;
   """)
   cur.execute("ALTER TABLE edges_custom ADD PRIMARY KEY (id);")
   cur.execute("ALTER TABLE edges_custom ADD COLUMN source integer;")
   cur.execute("ALTER TABLE edges_custom ADD COLUMN target integer;")
   conn.commit()
   print("✅ 'edges_custom' table created.")
else:
   print("ℹ️ 'edges_custom' table already exists.")

conn.commit()

ℹ️ 'edges_custom' table already exists.


In [16]:
# Check if edges_vertices_pgr table exists, if not create topology
cur.execute("""
   SELECT EXISTS (
      SELECT FROM information_schema.tables 
      WHERE table_name = 'edges_vertices_pgr'
   );
""")
vertices_exists = cur.fetchone()[0]

if not vertices_exists:
   print("--- Creating topology with pgr_createTopology ---")
   cur.execute("""SELECT pgr_createTopology('edges_custom', 0.001, 'geom', 'id');""")
   conn.commit()
   print("✅ Topology created. Node table: 'edges_custom_vertices_pgr'")
else:
   print("ℹ️ 'edges_custom_vertices_pgr' table already exists.")

conn.commit()

--- Creating topology with pgr_createTopology ---
✅ Topology created. Node table: 'edges_custom_vertices_pgr'


## 5. Run Shortest Path Queries (A* and Dijkstra)
Use `pgr_aStar` and `pgr_dijkstra` to find the shortest path between the start and end nodes. Compare the results and visualize both paths on the map.

In [17]:
# Start vertex
cur.execute("""
    SELECT id 
    FROM edges_custom_vertices_pgr 
    ORDER BY the_geom <-> ST_SetSRID(ST_MakePoint(%s, %s), 4326)
    LIMIT 1;
""", (start_lon, start_lat))
start_vid = cur.fetchone()[0]

# End vertex
cur.execute("""
    SELECT id 
    FROM edges_custom_vertices_pgr 
    ORDER BY the_geom <-> ST_SetSRID(ST_MakePoint(%s, %s), 4326)
    LIMIT 1;
""", (end_lon, end_lat))
end_vid = cur.fetchone()[0]


# Example usage of pgr_aStar for one-to-one shortest path calculation
# Using start_vid and end_vid (vertex IDs) instead of road IDs
query = '''
SELECT * FROM pgr_aStar(
  'SELECT id, source, target, ST_Length(geom::geography) AS cost, ST_Length(geom::geography) AS reverse_cost,
   ST_X(ST_StartPoint(geom)) AS x1, ST_Y(ST_StartPoint(geom)) AS y1,
   ST_X(ST_EndPoint(geom)) AS x2, ST_Y(ST_EndPoint(geom)) AS y2
   FROM edges_custom',
  %s, %s,
  directed => false, heuristic => 1);
'''
cur.execute(query, (start_vid, end_vid))
result = cur.fetchall()
if result:
    for row in result:
        print(row)
else:
    print("No path found")

(1, 1, 25254, 9788, 25254, 226262300, 92.16118745874509, 0.0)
(2, 2, 25254, 9788, 25372, 215047987, 93.77417115358605, 92.16118745874509)
(3, 3, 25254, 9788, 25360, 215048270, 216.01785058734885, 185.93535861233113)
(4, 4, 25254, 9788, 25357, 1380098351, 170.1902668309914, 401.95320919968)
(5, 5, 25254, 9788, 25409, 1376897355, 33.693752284883814, 572.1434760306713)
(6, 6, 25254, 9788, 25410, 683517846, 59.64660121114762, 605.8372283155551)
(7, 7, 25254, 9788, 25411, 226094898, 35.442564646594185, 665.4838295267027)
(8, 8, 25254, 9788, 25441, 1446063191, 70.5103617693074, 700.9263941732969)
(9, 9, 25254, 9788, 25407, 215048150, 157.23710745814617, 771.4367559426043)
(10, 10, 25254, 9788, 25439, 226084879, 42.157524312175426, 928.6738634007505)
(11, 11, 25254, 9788, 25440, 226084920, 20.217413599697895, 970.8313877129259)
(12, 12, 25254, 9788, 25433, 1153914558, 175.77880092497335, 991.0488013126238)
(13, 13, 25254, 9788, 25448, 675820461, 72.57962637894236, 1166.8276022375971)
(14, 14,

In [18]:
import folium
import json

# Create map centered at the start point
m = folium.Map(location=[start_lat, start_lon], zoom_start=13)

# --- A* Path ---
astar_edge_ids = [row[5] for row in result if row[5] != -1]
if astar_edge_ids:
    sql = f"SELECT ST_AsGeoJSON(geom) FROM edges_custom WHERE id IN ({','.join(map(str, astar_edge_ids))});"
    cur.execute(sql)
    geoms = [row[0] for row in cur.fetchall()]
    for geojson_str in geoms:
        coords = json.loads(geojson_str)["coordinates"]
        folium.PolyLine([(c[1], c[0]) for c in coords], color="red", weight=5, tooltip="A* Path").add_to(m)

# --- Dijkstra Path ---
dijkstra_query = """
SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, ST_Length(geom::geography) AS cost FROM edges_custom',
    %s, %s,
    directed := false
    );
"""
cur.execute(dijkstra_query, (start_vid, end_vid))
dijkstra_result = cur.fetchall()
dijkstra_edge_ids = [row[5] for row in dijkstra_result if row[5] != -1]
if dijkstra_edge_ids:
    sql = f"SELECT ST_AsGeoJSON(geom) FROM edges_custom WHERE id IN ({','.join(map(str, dijkstra_edge_ids))});"
    cur.execute(sql)
    geoms = [row[0] for row in cur.fetchall()]
    for geojson_str in geoms:
        coords = json.loads(geojson_str)["coordinates"]
        folium.PolyLine([(c[1], c[0]) for c in coords], color="blue", weight=3, tooltip="Dijkstra Path").add_to(m)

# Mark start and end points
folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color="green")).add_to(m)
folium.Marker([end_lat, end_lon], popup="End", icon=folium.Icon(color="purple")).add_to(m)

# Save map
m.save("shortest_path_comparison_map.html")
print("✅ Both A* and Dijkstra paths visualized: shortest_path_comparison_map.html")

✅ Both A* and Dijkstra paths visualized: shortest_path_comparison_map.html


In [19]:
conn.rollback()